<a href="https://colab.research.google.com/github/lili-alsh/Contract_work/blob/master/Contract.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Эта часть проверяет файл выгрузки события по договорам (БИТ), все ли поля правильно заполнены. 
# Выводит список событий договоров, которые заполнены не корректно.

import pandas as pd

# Читаем файл выгрузки с договорами
data = pd.read_excel("/content/drive/My Drive/Договорная работа/Выгрузка договоров.xlsx")

type_pmnt_to_date = ['Предоплата до конкретной даты', 'Расчет до конкретной даты']
type_pmnt_sched = ['Расчет по графику платежей']
req_adds = ['Оплата по договору', 'Спецификация / Приложение/ Заявка необходимы']
data = data[(data['Тип договора'].isna()) |
            (data['Предмет договора'].isna()) |
            ((data['Срок действия'].isna()) & (data['Вид расчетов'].isna())) |
            ((~data['Срок действия'].isna()) & (~data['Вид расчетов'].isna())) |
            ((~data['Вид события'].isin(type_pmnt_sched)) & (~data['Вид события'].isin(type_pmnt_to_date)) 
            & (data['Кол-во дней  после выполнения'].isna())) | 
            ((data['Вид события'].isin(type_pmnt_to_date)) & (data['Дата выполнения'].isna())) |
            ((~data['Кол-во дней  после выполнения'].isna()) & (~data['Дата выполнения'].isna())) |
            ((data['Процент'].isna()) & (data['Сумма'].isna()) & (~data['Вид события'].isin(type_pmnt_sched))) |
            ((~data['Процент'].isna()) & (~data['Сумма'].isna())) |
            (data['Комментарий'].isna()) | (data['Стадия договора'].isna()) |
            (data['Сумма договора'] == 10) | (~data['Комментарий'].isin(req_adds))]
data.to_excel('Договоры на проверку.xlsx', index=False)

In [ ]:
# ПОСЛЕ УСТРАНЕНИЯ НЕВЕРНОГО ЗАПОЛНЕНИЯ Событий договора (БИТ) выводим список договоров с поставщиками к закрытию.

import pandas as pd
from datetime import datetime

# Читаем файл выгрузки с договорами
data = pd.read_excel("/content/drive/My Drive/Договорная работа/Выгрузка договоров.xlsx")

# Берем текущую дату
curr_date = datetime.now()
# Приводим к формату даты
date = pd.to_datetime(data['Срок действия'], dayfirst=True)
data['Срок действия'] = date
# Добавляем данные по незакрытым со сроком действия меньше текущей даты
data_toclose = data[(data['Стадия договора'] != 'Закрыт') &
            (data['Срок действия'] < curr_date)]
# Убираем дубли
data_toclose = data_toclose.drop_duplicates(subset=['Код'])

type_cntr_suplly = ['Договор подряда или возмездного оказания услуг (расходный)', 'Договор купли-продажи (расходный)',
                    'Договор-счет, оферта (не требует подписания)', 'Договор аренды']

# Данные для закрытия договоров с поставщиками
data_supply = data_toclose[data_toclose['Тип договора'].isin(type_cntr_suplly)]

# Берем данные по договорам с поставщиками, которые будем сравнивать с ОСВ
data_OSV = data[data['Тип договора'].isin(type_cntr_suplly)]
data_OSV = data_OSV[(data_OSV['Вид расчетов'] == 'До момента исполнения обязательств') & (data_OSV['Стадия договора'] != 'Закрыт')]
# Читаем файл выгрузки ОСВ
OSV_60 = pd.read_excel("/content/drive/My Drive/Договорная работа/ОСВ 60.xlsx")
OSV_76 = pd.read_excel("/content/drive/My Drive/Договорная работа/ОСВ 76.xlsx")
OSV = OSV_60.merge(OSV_76, how = 'outer')
# Объединяем файл с ОСВ
data_turn = data_OSV.merge(OSV, how = 'left', left_on='Код',right_on='Договоры_Код')
# Выбираем договоры, по которым нулевое сальдо на конец
data_turn = data_turn[(data_turn['Сальдо_Дебет'].isna()) & (data_turn['Сальдо_Кредит'].isna())] 

# Выбираем договоры, по которым сумма оборотов за период больше или равна сумме договора
data_turn = data_turn[(data_turn['Обороты за период_Дебет'] > data_turn['Сумма договора']) | 
                      (data_turn['Обороты за период_Дебет'] == data_turn['Сумма договора'])]
# Убираем дубли
data_turn = data_turn.drop_duplicates(subset=['Код'])
# Убираем лишние столбцы
data_turn = data_turn.drop(columns=['Договоры_Код', 'Обороты за период_Дебет', 'Обороты за период_Кредит', 'Сальдо_Дебет', 'Сальдо_Кредит'])

# Объединяем договоры для закрытия по поставщикам
data_close = data_supply.merge(data_turn, how = 'outer')

# Сохраняем файл
data_close.to_excel('Договоры с поставщиками для закрытия.xlsx', index=False)

In [ ]:
# ПОСЛЕ УСТРАНЕНИЯ НЕВЕРНОГО ЗАПОЛНЕНИЯ Событий договора (БИТ) выводим список договоров с покупателями к закрытию.

import pandas as pd
from datetime import datetime

# Читаем файл выгрузки с договорами
data = pd.read_excel("/content/drive/My Drive/Договорная работа/Выгрузка договоров.xlsx")

# Берем текущую дату
curr_date = datetime.now()
# Приводим к формату даты
date = pd.to_datetime(data['Срок действия'], dayfirst=True)
data['Срок действия'] = date
# Добавляем данные по незакрытым со сроком действия меньше текущей даты
data_toclose = data[(data['Стадия договора'] != 'Закрыт') &
            (data['Срок действия'] < curr_date)]
# Убираем дубли
data_toclose = data_toclose.drop_duplicates(subset=['Код'])

type_cntr_suplly = ['Договор подряда или возмездного оказания услуг (расходный)', 'Договор купли-продажи (расходный)',
                    'Договор-счет, оферта (не требует подписания)', 'Договор аренды']

# Данные для закрытия договоров с покупателями
data_buy = data_toclose[~data_toclose['Тип договора'].isin(type_cntr_suplly)]

# Берем данные по договорам с покупателями, которые будем сравнивать с ОСВ
data_OSV = data[~data['Тип договора'].isin(type_cntr_suplly)]
data_OSV = data_OSV[(data_OSV['Вид расчетов'] == 'До момента исполнения обязательств') & (data_OSV['Стадия договора'] != 'Закрыт')]
# Читаем файл выгрузки ОСВ
OSV_62 = pd.read_excel("/content/drive/My Drive/Договорная работа/ОСВ 62.xlsx")
OSV_76 = pd.read_excel("/content/drive/My Drive/Договорная работа/ОСВ 76.xlsx")
OSV = OSV_62.merge(OSV_76, how = 'outer')
# Объединяем файл с ОСВ
data_turn = data_OSV.merge(OSV, how = 'left', left_on='Код',right_on='Договоры_Код')
# Выбираем договоры, по которым нулевое сальдо на конец
data_turn = data_turn[(data_turn['Сальдо_Дебет'].isna()) & (data_turn['Сальдо_Кредит'].isna())] 

# Выбираем договоры, по которым сумма оборотов за период больше или равна сумме договора
data_turn = data_turn[(data_turn['Обороты за период_Дебет'] > data_turn['Сумма договора']) | 
                      (data_turn['Обороты за период_Дебет'] == data_turn['Сумма договора'])]
# Убираем дубли
data_turn = data_turn.drop_duplicates(subset=['Код'])
# Убираем лишние столбцы
data_turn = data_turn.drop(columns=['Договоры_Код', 'Обороты за период_Дебет', 'Обороты за период_Кредит', 'Сальдо_Дебет', 'Сальдо_Кредит'])

# Объединяем договоры для закрытия по покупателям
data_close = data_buy.merge(data_turn, how = 'outer')

# Сохраняем файл
data_close.to_excel('Договоры с покупателями для закрытия.xlsx', index=False)

In [ ]:
# ПОСЛЕ ЗАКРЫТИЯ договоров обновляем реестр поставщиков

# import pandas as pd

# # Читаем файл выгрузки с договорами
# data = pd.read_excel("/content/drive/My Drive/Договорная работа/Выгрузка договоров.xlsx")

# type_cntr_suplly = ['Договор подряда или возмездного оказания услуг (расходный)', 'Договор купли-продажи (расходный)',
#                     'Договор-счет, оферта (не требует подписания)']

# # Отбираем действующие договоры с поставщиками
# data = data[(data['Тип договора'].isin(type_cntr_suplly)) &
#             (data['Стадия договора'] == 'Исполнение') & 
#             (((data['Комментарий'] == 'Оплата по договору') & (data['Сумма договора'].isna())) | 
#             (data['Комментарий'] == 'Спецификация / Приложение/ Заявка необходимы'))]
# # Убираем дубли
# data = data.drop_duplicates(subset=['Код'])

# # Читаем файл выгрузки реестра поставщиков
# reg = pd.read_excel("/content/drive/My Drive/Договорная работа/Реестр поставщиков.xlsx")

# # Обновляем срок действия в файле
# new_reg = reg.merge(data, how = 'left', left_on='Договор',right_on='Проект договора / Договор контрагента')
# new_reg = new_reg[(~new_reg['Срок действия_y'].isna()) | (~new_reg['Вид расчетов'].isna())]
# new_reg['Срок действия'] = new_reg[['Срок действия_y', 'Вид расчетов']].fillna('').sum(axis=1)
# new_reg = new_reg.rename(columns={'Контрагент_x':'Контрагент'})
# new_reg = new_reg[['Контрагент', 'ИНН', 'Плательщик НДС', 'Договор', 'Вид товаров / услуг / работ', 'История произведенных закупок',
#                    'Срок действия', 'Ответственное подразделение', 'Ссылка на договор']]

# add_reg = data.merge(reg, how = 'left', left_on='Проект договора / Договор контрагента',right_on='Договор')
# add_reg = add_reg[(add_reg['Контрагент_y'].isna()) & (add_reg['Контрагент_x'] != 'ИННОПОЛИС АО') &
#                   ((~add_reg['Проект договора / Договор контрагента'].str.contains('Камаз')) | 
#                    (~add_reg['Проект договора / Договор контрагента'].str.contains('ЦНТИ')) |
#                    (~add_reg['Проект договора / Договор контрагента'].str.contains('Камаз')) |
#                    (~add_reg['Проект договора / Договор контрагента'].str.contains('Конвертоплан')) |
#                    (~add_reg['Проект договора / Договор контрагента'].str.contains('ЦК'))) &
#                   (add_reg['Проект договора / Договор контрагента'].str.contains('2020'))]
# add_reg.to_excel('Для реестра.xlsx', index=False)